# Install datasets
Restart runtime after installing libraries.

In [ ]:
!pip install datasets
!pip install transformers
!pip install accelerate

# Download dataset and model
Download dataset from huggingface or get local dataset

In [2]:
# huggingface dataset
from datasets import load_dataset
dataset = load_dataset("tatsu-lab/alpaca")

In [ ]:
dataset

# Download model

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy

torch.manual_seed(42)
output_dir = "EleutherAI/gpt-neo-125m"
tokenizer = AutoTokenizer.from_pretrained(output_dir, bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = AutoModelForCausalLM.from_pretrained(output_dir, low_cpu_mem_usage=True, device_map="auto") # .cuda() torch_dtype=torch.float16,
model.resize_token_embeddings(len(tokenizer))

# Preprocess

In [ ]:
dataset = dataset['train']

In [19]:
data_list = []
text = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."
for i in range(50): # len(dataset['instruction'])):
    if dataset['input']:
        temp = text + "###Instruction:" + dataset['instruction'][i] + "\n\n###Input:" + dataset['input'][i] + "\n\n###Output:" + dataset['output'][i]
    else:
        temp = text + "###Instruction:" + dataset['instruction'][i] + "\n\n###Output:" + dataset['output'][i]
    data_list.append(temp)

In [20]:
data_list = pd.Series(data_list)

In [ ]:
data_list[0]

In [ ]:
len(data_list)

In [ ]:
max_length = max([len(tokenizer.encode(e)) for e in data_list])
print("Max length: {}".format(max_length))

In [24]:
class MyDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer(f"<|startoftext|>"+ txt + "<|endoftext|>", truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [25]:
mydataset = MyDataset(data_list, tokenizer, max_length = max_length)

In [ ]:
mydataset

# Train

In [27]:
import gc
import torch
from transformers import DataCollatorForLanguageModeling, AutoConfig, default_data_collator
gc.collect()
torch.cuda.empty_cache()

In [ ]:
training_args = TrainingArguments(
    output_dir = "finetune_save",
    optim="adafactor", # 메모리 사용 감소
    num_train_epochs=1,
    per_device_train_batch_size=2,
    # fp16=True,
    learning_rate=5e-5,
    logging_strategy="steps",
    warmup_steps=3000,
    weight_decay=0.01,
    logging_steps=0.1,
    save_steps=0.06, # 소수점은 퍼센트, 정수면 스텝수
    save_strategy="steps",
    report_to="none"
)

trainer = Trainer(model=model, args=training_args, train_dataset=mydataset,
    data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])})

trainer.train()

# Save model

In [ ]:
output_dir = "./FT/diff_tokenizer"

In [ ]:
# save as trainer
# tokenizer.save_pretrained(output_dir)
# trainer.save_model(output_dir)

In [ ]:
# save model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Load saved model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# tokenizer = AutoTokenizer.from_pretrained(output_dir)
# model = AutoModelForCausalLM.from_pretrained(output_dir, torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device=f"cuda", non_blocking=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained(output_dir)
# tokenizer = AutoTokenizer.from_pretrained("eunyounglee/GPT-NeoX-1.3B-Viet-finetune-epoch-10")
model = AutoModelForCausalLM.from_pretrained(output_dir).to("cuda")

# Load Dataset again

In [ ]:
# from datasets import load_dataset
# dataset = load_dataset("tatsu-lab/alpaca")
# dataset = dataset['train']

# Generate 1

In [ ]:
i = 0
text = "###Question:" + dataset['question'][i] + "\n###Answer:"

In [ ]:
text

In [ ]:
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "left"
generated = tokenizer("<|startoftext|>" + text, return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                            #     bos_token='<|startoftext|>',
                             #    eos_token='<|endoftext|>', pad_token='[PAD]', padding_side="left",
                                max_length=300, top_p=0.95, temperature=1.0, num_return_sequences=5)

In [ ]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {} \n".format(i, tokenizer.decode(
        sample_output, skip_special_tokens=True)))

# Generate 2

In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0) # device=0)

In [ ]:
prompt3 = text #@param {type:"string"}
response_min_chars =  20#@param {type:"integer"}
response_max_chars =  200#@param {type:"integer"}

response_3 = generator(prompt3, do_sample=True, min_length=response_min_chars,
                       max_length=response_max_chars,
                       clean_up_tokenization_spaces=True,
                       return_full_text=True)
out3_dict = response_3[0]
out3_dict

# Upload to Huggingface

In [ ]:
from huggingface_hub import login
login()

In [ ]:
tokenizer.push_to_hub("eunyounglee/GPT-NeoX-1.3B-viet-custom-10")

In [ ]:
model.push_to_hub("eunyounglee/GPT-NeoX-1.3B-viet-custom-10")